In [1]:
import json 
import pandas as pd
import numpy as np
import time
from scipy.sparse import dok_matrix
import itertools
from sklearn import preprocessing
import os

In [2]:
# importar varios json 
nombres = [i for i in os.listdir('./mpd.v1/data/') if i.endswith('.json')]

data_various = []
for i in nombres:
    data_various.append(json.load(open('./mpd.v1/data/{}'.format(i))))

In [3]:
data = json.load(open('./mpd.v1/data/mpd.slice.0-999.json'))

n_listas = 1000

In [4]:
lists_data = []

for i in range(0,n_listas):
    for x in data['playlists'][i]['tracks']:
        lists_data.append([data['playlists'][i]['name'],data['playlists'][i]['num_albums'], data['playlists'][i]['num_artists'],data['playlists'][i]['num_followers'],data['playlists'][i]['num_tracks'], data['playlists'][i]['pid'], x['pos'], x['artist_name'], x['track_name'], x['duration_ms'], x['album_name']])

df = pd.DataFrame(lists_data, columns = ['play_list_name', 'num_albums', 'num_artists', 'num_followers', 'num_tracks', 'pid', 'song_pos', 'song_artist', 'song_name', 'song_duration', 'song_album'])

In [5]:
df.head()

,play_list_name,num_albums,num_artists,num_followers,num_tracks,pid,song_pos,song_artist,song_name,song_duration,song_album
0,Throwbacks,47,37,1,52,0,0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),226863,The Cookbook
1,Throwbacks,47,37,1,52,0,1,Britney Spears,Toxic,198800,In The Zone
2,Throwbacks,47,37,1,52,0,2,Beyoncé,Crazy In Love,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Throwbacks,47,37,1,52,0,3,Justin Timberlake,Rock Your Body,267266,Justified
4,Throwbacks,47,37,1,52,0,4,Shaggy,It Wasn't Me,227600,Hot Shot


In [7]:
# listas de listas de canciones 
listas = {}
cont = 0 

song_index = {}
index_song = {}
cont_song = 0

# cambiar 'l' a "data" y quitar esta iteracion si queremos hacerlo con 1 json  
for l in data_various[0:1]:
    for i in range(0,n_listas):

        listas[l['playlists'][i]['pid']] = [x['track_name'] for x in l['playlists'][i]['tracks']]
        for x in listas[l['playlists'][i]['pid']]:
            if not x in song_index:
                song_index[x] = cont_song
                index_song[cont_song] = x
                cont_song += 1


In [8]:
len(song_index)

30172

In [ ]:
inicio = time.time()
size = len(song_index)

matriz = dok_matrix((size,size),dtype='uint')

for lista in listas:
    for (i,j) in itertools.combinations(listas[lista],2):
        if i!=j:
            matriz[song_index[i],song_index[j]] += 1
               
fin = time.time()

print('tiempo demorado: {} segs'.format(fin - inicio))

In [19]:
now = time.time()
result = matriz.toarray()
pos = np.where(result > 10)

print('tiempo demorado: {} segs'.format(time.time() - now))

tiempo demorado: 86.57010507583618 segs


In [20]:
pairs = []
in_pair = set()
for x,y in zip(pos[0], pos[1]):
    pairs.append([x,y,result[x,y]])
    in_pair.add(x)
    in_pair.add(y)

In [21]:
pd.DataFrame(pairs, columns = ['Source','Target', 'Weight']).to_csv('spotify_graph.csv', sep=';', index= False)

In [22]:
pairs_2 = [[x,index_song[x]] for x in in_pair]

In [23]:
pd.DataFrame(pairs_2, columns = ['id', 'label']).to_csv('spotify_labels.csv', sep=';', index=False)